## Display locations of dense [AIS broadcast reports](https://marinecadastre.gov/ais/) at the port of Miami, FL.

Import the `GIS` module from the [ArcGIS API for Python](https://developers.arcgis.com/python/)

In [ ]:
from arcgis.gis import GIS

Get a reference to the underlying JVM

In [ ]:
jvm = sc._jvm

Get a reference to the `FileGDB` Scala object

In [ ]:
gdb = jvm.com.esri.gdb.FileGDB

Print the list of tables in the geo database:

In [ ]:
tables = gdb.listTables('Miami.gdb')
for t in tables:
    print(t)

Print the schema of `Broadcast` (note how `x` and `y` are subfields to `Shape`)

In [ ]:
schema = gdb.schema('Miami.gdb','Broadcast')
print(schema.treeString())

Create a Spark Dataframe and register it as a table for upcoming SQL operations

In [ ]:
df = spark.read \
    .format("com.esri.gdb") \
    .options(path="Miami.gdb", name="Broadcast") \
    .load()

In [ ]:
df.registerTempTable("Broadcast")

Create temp table `QR` by mapping a `Broadcast` event to a cell location.  The cell size is 0.001 degrees.

In [ ]:
sql("select cast(floor(Shape.x/0.001) as INT) as q,cast(floor(Shape.y/0.001) as INT) as r from Broadcast")\
.registerTempTable("QR")

Aggregate by cell and report back only the cells with more that 100 `Broadcast` events.
Map each cell q/r to an x/y geo location and convert the Spark dataframe into a Pandas dataframe.

In [ ]:
pdf = sql("select q*0.001+0.0005 as x,r*0.001+0.0005 as y,count(1) as pop from QR group by q,r having pop > 100")\
.toPandas()

Create a `dark-gray` map centered around Miami, FL.

In [ ]:
gis = GIS()
m = gis.map('Miami, FL')
m.basemap = 'dark-gray'
m

Add to the map a layer whose content is imported from a Pandas dataframe.

In [ ]:
m.add_layer(gis.content.import_data(pdf))